In [ ]:
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
path = "/content/drive/MyDrive/Pratikum_Ml/pratikum03/"

In [ ]:
#membaca file csv
import pandas as pd
df = pd.read_csv(path + "data/socr.csv", sep=',')
df.head ()

In [ ]:
df.describe()

#konversi data menjadi satuan cm dan kg

In [ ]:
df1 = (
    df[["Height(Inches)", 'Weight(Pounds)']]
    .rename(columns={"Height(Inches)": "tinggi_cm","Weight(Pounds)":"berat_kg"})
    .assign(
        tinggi_cm = lambda d: d["tinggi_cm"] * 2.54,
        berat_kg = lambda d: d["berat_kg"] * 0.453592
    )
    .round({"tinggi_cm":2,"berat_kg":2})
).copy()
df1.head()

membagi dataset untuk training dan test

In [ ]:
from sklearn.model_selection import train_test_split
x=df1[["tinggi_cm"]]
y=df1[["berat_kg"]]
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.2, random_state=7)
x_train.head()


In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#lakukan proses training
model.fit(x_train, y_train)


Evaluasi Model



In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

y_pred=model.predict(x_test)
r2 = r2_score(y_test, y_pred)

df_resume = pd.DataFrame()
df_resume["R2"]= r2
df_resume["koefisien (kg/cn)"]= model.coef_[0]
df_resume["intersip (kg)"]= model.intercept_
df_resume["MAE (kg)"]= mean_absolute_error(y_test, y_pred)
mse= mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
df_resume["MSE (kg)"] =mse
df_resume["RMSE (kg)"] =rmse
df_resume


In [ ]:
slope = model.coef_.item()
intercept = model.intercept_.item()
print(f"persamaan: y= {slope:.3f} * x + {intercept:.3f}")

In [ ]:
import matplotlib.pyplot as plt

#plot data scotter
plt.figure(figsize=(8, 5))
plt.scatter(x_test, y_test, color="blue", label="Data aktual")
plt.plot(x_test, y_pred,color="red", linewidth=2, label="Prediksi")
plt.xlabel("tinggi (cm)")
plt.ylabel("berat (kg)")
plt.title("Regresi Linear: Berat vs Tinggi")
plt.text(
    0.02, 0.98,
    f"y={slope:.3f}*x+{intercept:.3f}\n$R^2$ = {r2:.3f}",
    transform=plt.gca().transAxes,va="top"
)
plt.legend()
plt.tight_layout()
plt.show()

data testing dari mahasiswa ti2025

In [ ]:
import pandas as pd
df2 = pd.read_csv("/content/drive/MyDrive/Pratikum_Ml/pratikum03/data/berat_ti2025.csv", sep=";")

y_pred = model.predict(df2[['tinggi_cm']])
df2['prediksi_kg'] = y_pred
df2['selisih error (kg)'] = df2['prediksi_kg'] - df2['actual_berat']
demon = df2['actual_berat'].replace(0, np.nan)
df2["akurasi (%)"] = (1 - (df2["selisih error (kg)"].abs() / demon)).clip(lower=0, upper=1) * 100
df2.head()

# **MULTIPLE LINEAR REGRESI**


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd

df = pd.read_csv("/content/gdrive/MyDrive/Pratikum_Ml/pratikum03/data/stunting_wasting_dataset.csv", sep=",")
df.head()

In [ ]:
df.describe()

In [ ]:
  df1 = (df[["Berat Badan (kg)", "Jenis Kelamin", "Umur (bulan)", "Tinggi Badan (cm)"]]
.rename(columns={"Jenis Kelamin":"jk", "Umur (bulan)":"umur_bln",
                 "Tinggi Badan (cm)":"tinggi_cm", "Berat Badan (kg)":"berat_kg"}).copy())

## Laki-laki: 1, Perempuan : 0
df1["jk"] = df1["jk"].map({"Laki-laki": 1, "Perempuan": 0})
df1.head()

In [ ]:
corr_matrix = df1.corr()

print(corr_matrix)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matrix Korelasi DF1")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

y = df1["berat_kg"]

X = df1[["umur_bln", "tinggi_cm"]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42 # random_state supaya hasil konsisten
)

print("Jumlah data train :", len(X_train))
print("Jumlah data test :", len(X_test))

X_train.head()

In [ ]:
import statsmodels.api as sm

X_train_const = sm.add_constant(X_train)
X_train_const.head()


In [ ]:
import statsmodels.api as sm


model = sm.OLS(y_train, X_train_const).fit()
print('----------------------------------------')
print(model.params)
print('----------------------------------------')
const = model.params['const']
x1_umur = model.params['umur_bln']
x2_tinggi = model.params['tinggi_cm']
print(f"y = {const:.3f} + {x1_umur:.3f}*x1 + {x2_tinggi:.3f}*x2")

In [ ]:
print(model.summary())

In [ ]:
import numpy as np

X_test_const = sm.add_constant(X_test)
y_pred_test = model.predict(X_test_const)

hasil = pd.DataFrame({
    "Umur (bulan)": X_test["umur_bln"].to_numpy(),
    "Tinggi (cm)": X_test["tinggi_cm"].to_numpy(),
    "Berat Aktual (kg)": y_test.to_numpy(),
    "Berat Prediksi (kg)": y_pred_test
})

hasil["Selisih error (kg)"] = hasil["Berat Prediksi (kg)"] - hasil["Berat Aktual (kg)"]

denom = hasil["Berat Aktual (kg)"].replace(0, np.nan)
hasil["Akurasi (%)"] = (1 - (hasil["Selisih error (kg)"].abs() / denom)).clip(lower=0, upper=1) * 100

hasil